In [1]:
%load_ext autoreload
%autoreload 2

This notebook imports the fundamental objects of the streamm package and goes through the functionality of each

Let's start with the Particle object 

In [2]:
from streamm.structures.particle import Particle

Create a particle object with label 'C1'

In [3]:
p_i = Particle(label='C1')

In [4]:
print(p_i)

atom:C1


Assign the carbon element to the particle

In [5]:
p_i.set_element('C')

Check that the element properties were set to the particle 

In [6]:
property_msg = " type:{} ".format(p_i.type)
property_msg += "\n label:{}".format(p_i.label)
property_msg += "\n symbol:{}".format(p_i.symbol)
property_msg += "\n mass:{}".format(p_i.mass)
property_msg += "\n charge:{}".format(p_i.charge)
property_msg += "\n bonded_radius:{}".format(p_i.bonded_radius)
property_msg += "\n nonbonded_radius:{}".format(p_i.nonbonded_radius)

print(property_msg)

 type:atom 
 label:C1
 symbol:C
 mass:12.0107 amu
 charge:0.0
 bonded_radius:0.67
 nonbonded_radius:1.7


Let's create another particle and set the element to hydrogen 

In [7]:
p_j = Particle(symbol='H')

In [8]:
property_msg = " type:{} ".format(p_j.type)
property_msg += "\n label:{}".format(p_j.label)
property_msg += "\n symbol:{}".format(p_j.symbol)
property_msg += "\n mass:{}".format(p_j.mass)
property_msg += "\n charge:{}".format(p_j.charge)
property_msg += "\n bonded_radius:{}".format(p_j.bonded_radius)
property_msg += "\n nonbonded_radius:{}".format(p_j.nonbonded_radius)

print(property_msg)

 type:atom 
 label:H
 symbol:H
 mass:1.00794 amu
 charge:0.0
 bonded_radius:0.53
 nonbonded_radius:1.2


Let's make an empty structure container 

In [9]:
from streamm.structures.container import Container

In [10]:
mol_i = Container('methane')

Now let's construct a molecule 

We can add the carbon at the origin using the ``add_partpos()`` function. 

In [11]:
pos_i = [0.0,0.0,0.0]
mol_i.add_partpos(p_i,pos_i)

In [12]:
print("Now the structure container has {} particle ".format(mol_i.n_particles))

Now the structure container has 1 particle 


Find the positions of the hydrogens to give a tetrahedral molecular geometry  with a little math

In [13]:
import numpy as np
import decimal

In [14]:
bond_length = float(decimal.Decimal(str(p_i.bonded_radius + p_j.bonded_radius)))

In [15]:
bond_length

1.2

In [16]:
tet_a = bond_length/np.sqrt(3)

In [17]:
print tet_a

0.692820323028


Add hydrogens

In [18]:
pos_j = [tet_a,tet_a,tet_a]
mol_i.add_partpos(p_j,pos_j)

We can add the subsequent hydrogens using the same particle object since add_partpos makes a deepcopy of the object when adding to the structure container

In [19]:
pos_j = [-tet_a,-tet_a,tet_a]
mol_i.add_partpos(p_j,pos_j)

In [20]:
pos_j = [-tet_a,tet_a,-tet_a]
mol_i.add_partpos(p_j,pos_j)

In [21]:
pos_j = [tet_a,-tet_a,-tet_a]
mol_i.add_partpos(p_j,pos_j)

Check the position array 

In [22]:
print mol_i.positions

[array([ 0.,  0.,  0.]), array([ 0.69282032,  0.69282032,  0.69282032]), array([-0.69282032, -0.69282032,  0.69282032]), array([-0.69282032,  0.69282032, -0.69282032]), array([ 0.69282032, -0.69282032, -0.69282032])]


The particles instance variable of the structure container is a dictionary, so we can just loop over that using the iteritems() function. 

In [23]:
for p_index,particle_i in mol_i.particles.iteritems():
    print p_index,particle_i

0 atom:C1
1 atom:H
2 atom:H
3 atom:H
4 atom:H


Hum, let's fix the labels of the hydrogens...

In [24]:
h_cnt = 1
for p_index,particle_i in mol_i.particles.iteritems():
    if( particle_i.symbol == 'H' ):
        particle_i.label = 'H{}'.format(h_cnt)
        h_cnt += 1
        

In [25]:
for p_index,particle_i in mol_i.particles.iteritems():
    print p_index,particle_i

0 atom:C1
1 atom:H1
2 atom:H2
3 atom:H3
4 atom:H4


Okay, that looks better

Print .xyz file and check geometry with a molecular viewer such as  Avogadro (https://avogadro.cc/) 

In [26]:
mol_i.write_xyz()

Looks good, you should have the geometry of a methane molecule with bond length of 1.2 Angstroms 

However, we have not told streamm about the bonds. There are a few ways to do this, let's do it explictly with the Bond object fist.

In [27]:
from streamm.structures.bond import Bond

based on the particle index values

In [28]:
b_ij = Bond(0,1)

Now add the bond to the bonds dictionary in the structure container

In [29]:
mol_i.add_bond(b_ij)

In [30]:
print("Now the structure container has {} particle/s and {} bond/s".format(mol_i.n_particles,mol_i.n_bonds))


Now the structure container has 5 particle/s and 1 bond/s


Neat, but adding all the bonds, bond angles and dihedrals explicitly would be pretty tedious, so let's use some functions to do that. 

First let's guess the ``bonded_nblist`` of the molecule based on the ``bonded_radius`` of each particle (atom)

In [31]:
mol_i.bonded_nblist = mol_i.guess_nblist(0,radii_buffer=1.25)

In [32]:
print mol_i.bonded_nblist

 NBlist of 5 particle with 8 connections


Let's take a look at the neighbor lists ``list`` and ``index`` instance variables 

In [33]:
print mol_i.bonded_nblist.list 
print mol_i.bonded_nblist.index 

[1, 2, 3, 4, 0, 0, 0, 0]
[0, 4, 5, 6, 7, 8]


Looking at the ``index `` for particle 0, we get that it has neighbors in the ``list`` from 0:3 (index[0]:index[0+1]-1). There for we know particle 0 has [1, 2, 3, 4] for nieghbors.

In [34]:
print mol_i.bonded_nblist.calc_nnab(0)

4


Now we can used the bonded neighbor list to construct the bonds,bond angles and dihedrals 

In [35]:
mol_i.bonded_bonds()
mol_i.bonded_angles()
mol_i.bonded_dih()


In [36]:
property_msg = " n_particles:{} ".format(mol_i.n_particles)
property_msg += "\n n_bonds:{}".format(mol_i.n_bonds)
property_msg += "\n n_angles:{}".format(mol_i.n_angles)
property_msg += "\n n_dihedrals:{}".format(mol_i.n_dihedrals)
property_msg += "\n n_impropers:{}".format(mol_i.n_impropers)

print(property_msg)

 n_particles:5 
 n_bonds:4
 n_angles:6
 n_dihedrals:0
 n_impropers:0


A little easier than adding everything by hand